We are going to explore retrieval augmented generation from start to finish.

In [ ]:
from tqdm.auto import tqdm
from time import sleep

In [54]:
import json
import pandas as pd

import google.generativeai as palm
from langchain.text_splitter import RecursiveCharacterTextSplitter
from IPython.display import display, Markdown

In [ ]:
from uuid import uuid4

In [ ]:
# Custom modules
from chunker import get_chunks
from embeddings_palm import get_palm_embeddings

### Data Preprocessing

In [20]:
df = pd.read_csv("data/Why Revolutions Fail.csv")
data = list(df.text)

### Embeddings

In [23]:
texts = get_chunks(data)

In [ ]:
chunks = []
for text in tqdm(texts):
    
    chunks.append(
        {
            'id': str(uuid4()),
            'values': get_palm_embeddings(text),
            'metadata': {
                'text': text
                }
        }
    )

#### Saving Embeddings

In [44]:
with open("data/text_embeddings_101.json", "w", encoding='utf-8') as f:
    json.dump(chunks, f, indent=2)

#### Loading Embeddings

In [5]:
# # Reading from JSON
# with open("data/text_embeddings_101.json") as json_file:
#     chunks = json.load(json_file)

## Pinecone

In [26]:
import os
import pinecone

### Credentials

In [27]:
pinecone_api_key = os.getenv('PINECONE_API_KEY')

### Creating an Index

In [45]:

index_name = 'general-rag-idx'

# initialize connection to pinecone
pinecone.init(
    api_key= pinecone_api_key,
    environment="gcp-starter"  # next to API key in console
)

In [46]:
# check if index already exists (it shouldn't if this is first time)
if index_name not in pinecone.list_indexes():
    # if does not exist, create index
    pinecone.create_index(
        index_name,
        dimension=len(chunks[0]['values']),
        metric='dotproduct'
    )
# connect to index
index = pinecone.GRPCIndex(index_name)
# view index stats
index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 0}},
 'total_vector_count': 0}

### Populating the Index

In [47]:
index.upsert(vectors=chunks)

upserted_count: 101

### Retrieval

#### Create an Embedding for the Query
This comes in the form of a query vector we will name `xq`.

In [48]:
query = "How do revolutions generally unfold?"

xq = get_palm_embeddings(query)


#### Query Vector Database

In [50]:
res = index.query(xq, top_k=5, include_metadata=True)
res

{'matches': [{'id': 'd91ba5c7-9762-4c3b-9736-23af509fa137',
              'metadata': {'chunk': 61.0,
                           'text': 'And if you’re not careful, it can become a '
                                   'raging inferno that destroys everything in '
                                   'its path.You will act upon the throne, but '
                                   'the throne will act upon you. But... '
                                   "you'll be different this time. 😈Those who "
                                   'start revolutions rarely end up in power.\n'
                                   "It's always those who bring the revolution "
                                   "to a conclusion.You need to read Lenin's "
                                   'Notes of a publicist.I’m so glad you made '
                                   'these videos on revolution. I’ve been '
                                   'working on this story set during the '
                            

## Retreival Augmented Generation

### Augmentation

In [51]:
# get list of retrieved text
contexts = [item['metadata']['text'] for item in res['matches']]

augmented_query = "\n\n---\n\n".join(contexts)+"\n\n-----\n\n"+query

### Primer Template

In [53]:
# system message to 'prime' the model
primer = f"""You are Q&A bot. A highly intelligent system that answers
user questions based on the information provided by the user above
each question. If the information can not be found in the information
provided by the user you truthfully say "I don't know".
"""

In [56]:
res = palm.chat(context=primer, messages=augmented_query)

In [58]:
display(Markdown(res.last))

'Revolutions are complex events that can unfold in many different ways. However, there are some general patterns that can be observed.\n\nFirst, revolutions often begin with a period of social unrest. This can be caused by a number of factors, such as economic inequality, political oppression, or ethnic conflict. When people feel that they are being denied their basic rights or that their government is not representing their interests, they may become more likely to support a revolution.\n\nSecond, revolutions often require a charismatic leader to emerge. This leader can provide a vision for the future and inspire people to take action. The leader must also be able to build a coalition of supporters and organize them to achieve their goals.\n\nThird, revolutions often require the use of violence. This is because the government will not simply give up power without a fight. The revolutionaries must be willing to use force to overthrow the government and establish a new order.\n\nFourth,